<a href="https://colab.research.google.com/github/8Raouf24/PFE/blob/master/benchmarkRS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install surprise

     |████████████████████████████████| 11.8MB 346kB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp36-cp36m-linux_x86_64.whl size=1670932 sha256=791b7c8e7583bf5a375228682729c27e32a30455b8957e27a9bfdb71b99ca72a
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [18]:
import os
import json
import gzip
import pandas as pd
from urllib.request import urlopen
from surprise import BaselineOnly
from surprise import Dataset
from surprise import Reader
from surprise import NormalPredictor,SVD

from surprise.model_selection import cross_validate

D'abord , nous récupérons notre dataset grace a la commande wget

In [7]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Musical_Instruments_5.json.gz

--2020-07-25 17:26:50--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Musical_Instruments_5.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 39356138 (38M) [application/octet-stream]
Saving to: ‘Musical_Instruments_5.json.gz’

Musical_Instruments 100%[===================>]  37.53M  28.1MB/s    in 1.3s    

2020-07-25 17:26:52 (28.1 MB/s) - ‘Musical_Instruments_5.json.gz’ saved [39356138/39356138]



Nous pouvons manipuler les données sous forme de générateur , que nous utiliserons pour remplir un dictionnaire ( les données sont sous format .json)


In [8]:
### load the meta data

data = []
with gzip.open('Musical_Instruments_5.json.gz') as f:
    for l in f:
        data.append(json.loads(l.strip()))
    
# total length of list, this number equals total number of products
print(len(data))

# first row of the list
print(data[0])

231392
{'overall': 5.0, 'verified': True, 'reviewTime': '10 30, 2016', 'reviewerID': 'A3FO5AKVTFRCRJ', 'asin': '0739079891', 'reviewerName': 'francisco', 'reviewText': "It's good for beginners", 'summary': 'Five Stars', 'unixReviewTime': 1477785600}


Notre pré-traitement consiste a récolter seulement les colonnes de notre DataFrame qui nous serviront a la recommandation , c-à-d les items , les users , et les ratings

In [13]:
# we create a new data frame containing the required informations for our benchmark

df = pd.DataFrame.from_dict(data)
min_rating = 1.0
max_rating = 5.0
data_dict = {'userID':df['reviewerID'],'itemID':df['asin'],'rating':df['overall'].apply(lambda x: (x - min_rating) / (max_rating - min_rating))}
df_rs = pd.DataFrame.from_dict(data_dict)



print(df_rs.info)

<bound method DataFrame.info of                 userID      itemID  rating
0       A3FO5AKVTFRCRJ  0739079891    1.00
1       A3UCGC1DHFMBCE  0739079891    1.00
2       A2S9SLRYLPGYZB  0739079891    1.00
3       A15RTJWPG8OKOE  0739079891    0.75
4       A12ET1WO3OAVU7  0739079891    0.00
...                ...         ...     ...
231387  A2GLR2Q3M1PC7P  B01HIDOPP2    1.00
231388  A3S1LSTRW57BP9  B01HIDOPP2    1.00
231389  A2BNWRRADDBSVO  B01HIDOPP2    0.75
231390  A1KRL5ZRON6DPT  B01HIDOPP2    0.00
231391  A3QN3W0PJ1DXIT  B01HIDOPP2    1.00

[231392 rows x 3 columns]>


In [14]:
reader = Reader(rating_scale=(0, 1))
data = Dataset.load_from_df(df_rs, reader)


Nous effectuons notre cross-validation

In [20]:
dic_results = cross_validate(SVD(), data, cv=5)
print(dic_results)

{'test_rmse': array([0.23916506, 0.23614123, 0.23593652, 0.23845521, 0.2376587 ]), 'test_mae': array([0.16501468, 0.16344699, 0.16328885, 0.16419601, 0.16355954]), 'fit_time': (12.309168100357056, 12.350298881530762, 12.432934045791626, 12.397401809692383, 12.321861267089844), 'test_time': (0.41034364700317383, 0.5963470935821533, 0.580794095993042, 0.4003767967224121, 0.5956194400787354)}


In [21]:
dic_results['test_rmse'].mean()

0.23747134256158828